In [7]:
import os
import pandas as pd
from tqdm import tqdm
import pandas as pd
import psutil
import os
import pandas as pd
from tqdm import tqdm
import multiprocessing
import time
import os
import shutil
import os
import pandas as pd
import h3
import random
from tqdm import tqdm
from collections import defaultdict
import multiprocessing
import pickle
import psutil

In [8]:
memory_info = psutil.virtual_memory()

print(f"Total Memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used Memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Percentage Used: {memory_info.percent}%")

Total Memory: 31.36 GB
Available Memory: 11.71 GB
Used Memory: 19.21 GB
Percentage Used: 62.7%


In [9]:
def memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 ** 3)  

print(f"Memory usage before loading: {memory_usage()} GB")

Memory usage before loading: 2.3350868225097656 GB


In [39]:
import os
import pandas as pd
from tqdm import tqdm
input_directory_path = '/disk_5/grid_v2_vm_data/disk_4/pre_processed_data/state_grp_PUNJAB/split_train_test_valid_state_PUNJAB/'
output_combined_csv_path = '/disk_4/uddeshya/Dataset/PUNJAB_full_5_address5_without_city.csv'

columns_to_keep = ['city', 'pincode', 'cleaned_address', 'lookup_lat', 'lookup_lng', 'no_of_delivereies']

def load_and_process_file(filename):
    file_path = os.path.join(input_directory_path, filename)
    df = pd.read_csv(file_path, usecols=columns_to_keep)
    #df['cleaned_address'] = df['cleaned_address'] + " " + df['city']
    df_filtered = df[df['no_of_delivereies'] >= 5]
    df_filtered = df_filtered.drop_duplicates(subset=['cleaned_address'])
    df_filtered = df_filtered.dropna()
    
    return df_filtered

def combine_csv_files():
    csv_files = [f for f in os.listdir(input_directory_path) if f.endswith('_full_train.csv')]
    
    combined_df = pd.DataFrame(columns=columns_to_keep)
    for filename in tqdm(csv_files, desc="Loading and processing CSV files"):
        df_filtered = load_and_process_file(filename)
        combined_df = pd.concat([combined_df, df_filtered])
    

    combined_df = combined_df.dropna()
    combined_df.to_csv(output_combined_csv_path, index=False)
    
    print(f"Combined CSV file saved to {output_combined_csv_path}")

if __name__ == "__main__":
    combine_csv_files()

Loading and processing CSV files:   0%|          | 0/420 [00:00<?, ?it/s]

Loading and processing CSV files: 100%|██████████| 420/420 [02:05<00:00,  3.36it/s]


Combined CSV file saved to /disk_4/uddeshya/Dataset/PUNJAB_full_5_address5_without_city.csv


In [32]:
file_path_10 = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_10.csv'
file_path_6 = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_5.csv'
file_path_05 = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_05.csv'
file_path_3 = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB.csv'

file_path_valid = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_valid_PUNJAB_full_1.csv'
file_path_test = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_test_PUNJAB_full_1.csv'

file_path_full = '/disk_4/uddeshya/Dataset/PUNJAB_full_5_address5_without_city.csv'

In [66]:
triplet_df = pd.read_csv(file_path_3)


In [62]:
len(triplet_df) ##10

282737

In [64]:
len(triplet_df) ##05

1172900

In [67]:
len(triplet_df) ##03

1627337

In [10]:
df_valid = pd.read_csv(file_path_valid)

In [11]:
len(df_valid)

107113

In [12]:
df_test = pd.read_csv(file_path_test)

In [13]:
len(df_test)

204872

In [40]:
df_full = pd.read_csv(file_path_full)

In [34]:
len(df_full) ## more than 1 without city!

7438504

len(df_full) 

In [36]:
df_full['cleaned_address'][:10].values

array([' 0 khosa pando',
       ' 0 vpo droli bhai near govt school and bus stand',
       ' 00 baghela wala', ' 00 kmal patti dagru moga',
       ' 00 lala lajpat rai college of engineering and technology',
       ' 00 mirza patti', ' 001 jhandiana garbi moga subdistrict',
       ' 001 moga subdistrict', ' 001 vpo nidhanwala near salina road',
       ' 01 daroli bhai near safu walaa road'], dtype=object)

In [37]:
addresses = df_full['cleaned_address'].tolist()
lat_lngs = df_full[['lookup_lat', 'lookup_lng']].values.tolist()

In [38]:
output_pickle_path = '/disk_4/uddeshya/Dataset/PUNJAB_1_for_lookup_without_city.pkl'

with open(output_pickle_path, 'wb') as f:
    pickle.dump({
        'addresses': addresses,
        'lat_lngs': lat_lngs
    }, f)

print(f"Triplets saved to {output_pickle_path}")

Triplets saved to /disk_4/uddeshya/Dataset/PUNJAB_1_for_lookup_without_city.pkl


In [14]:
df_test.head()

city  pincode  \
0           Pathankot   145025   
1  Pathankot District   145025   
2  Pathankot District   145025   
3           Pathankot   145025   
4           Pathankot   145025   

                                     cleaned_address  lookup_lat  lookup_lng  \
0                       0 adda barth sahib Pathankot   32.244419   75.567518   
1   0 barth sahib puli radha swami pawan Pathanko...   32.244132   75.568270   
2   0 old route no nh 15 pathankot subdistrict Pa...   32.244134   75.568292   
3              1 farwaal post office sarna Pathankot   32.259957   75.573422   
4                           1 lahri bawian Pathankot   32.258106   75.545563   

   no_of_delivereies  
0                 21  
1                 36  
2                 30  
3                 20  
4                 29

In [55]:
triplet_df.head()

city  pincode                            cleaned_address  \
0           Moga   142048                         1 mothan wali Moga   
1           Moga   142048    103 village saffuwala jaittu patti Moga   
2           Moga   142048       118 village mangewala dist moga Moga   
3  Moga District   142048         131 moga subdistrict Moga District   
4           Moga   142048   132 vpo dagru moga moga subdistrict Moga   

   lookup_lat  lookup_lng  no_of_delivereies  
0   30.782533   75.098455                 10  
1   30.791183   75.079569                 11  
2   30.789038   74.974681                 14  
3   30.831992   75.015384                 10  
4   30.831935   75.049449                 13

In [56]:
len(triplet_df)

282737

In [57]:
triplet_df['cleaned_address'][1167]

'guru ravidas gurudwara village khatti houshiarpur road Phagwara'

In [58]:
# Display basic information about the DataFrame
print("Basic Information:")
print(triplet_df.info())

# Check for NaN values
print("\nNaN Values:")
print(triplet_df.isna().sum())

# Check for duplicates
duplicates = triplet_df.duplicated()
num_duplicates = duplicates.sum()
print(f"\nNumber of duplicate rows: {num_duplicates}")

if num_duplicates > 0:
    print("\nDuplicate rows:")
    print(triplet_df[duplicates])



Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282737 entries, 0 to 282736
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   city               282737 non-null  object 
 1   pincode            282737 non-null  int64  
 2   cleaned_address    282737 non-null  object 
 3   lookup_lat         282737 non-null  float64
 4   lookup_lng         282737 non-null  float64
 5   no_of_delivereies  282737 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 12.9+ MB
None

NaN Values:
city                 0
pincode              0
cleaned_address      0
lookup_lat           0
lookup_lng           0
no_of_delivereies    0
dtype: int64

Number of duplicate rows: 0


In [48]:
print("\nSummary Statistics:")
print(triplet_df.describe())

print("\nFirst few rows of the DataFrame:")
print(triplet_df.head())


Summary Statistics:
             pincode     lookup_lat     lookup_lng  no_of_delivereies
count  878857.000000  878857.000000  878857.000000      878857.000000
mean   146051.253735      30.965509      75.781288           8.606694
std      5843.081878       0.571127       0.687864           2.410087
min    140001.000000      29.666541      73.886877           6.000000
25%    141016.000000      30.613878      75.367734           7.000000
50%    144023.000000      30.862378      75.812500           8.000000
75%    148028.000000      31.362652      76.391477          10.000000
max    176023.000000      32.387450      77.091570          14.000000

First few rows of the DataFrame:
            city  pincode                                    cleaned_address  \
0  Moga District   142048                      00 baghela wala Moga District   
1           Moga   142048               02 isf college of pharmacy moga Moga   
2           Moga   142048           1 dhalle_ke br canda zira road moga Mog

In [49]:
memory_usage = triplet_df.memory_usage(deep=True)
total_memory_usage = memory_usage.sum()

print("Memory Usage by Column:")
print(memory_usage)
print(f"\nTotal Memory Usage: {total_memory_usage / (1024 ** 2):.2f} MB")

Memory Usage by Column:
Index                      128
city                  58080628
pincode                7030856
cleaned_address      103232598
lookup_lat             7030856
lookup_lng             7030856
no_of_delivereies      7030856
dtype: int64

Total Memory Usage: 180.66 MB


In [64]:
combined_csv_path = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB.csv'
output_pickle_path = '/disk_4/uddeshya/Dataset/combined_triplets.pkl'

def generate_h3_cells(lat_lngs, resolution):
    return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

def is_valid_h3_cell(cell):
    try:
        h3.h3_get_resolution(cell)
        return True
    except h3.H3CellError:
        return False

def generate_triplets_for_cell(cell, points, h3_cells, K, T, D, resolution):
    triplets = []
    if len(points) > 1:
        current_resolution = h3.h3_get_resolution(cell)
        if current_resolution >= resolution:
            try:
                parent_cell = h3.h3_to_parent(cell, resolution - 1)
            except (h3.H3CellError, h3.H3ResolutionError):
                return triplets

            rings = []
            for ring_level in range(1, D + 1):
                if ring_level == 1:
                    ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1)
                else:
                    previous_ring = h3.k_ring(parent_cell, ring_level)
                    ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
                rings.append(ring)

        valid_negative_samples_by_level = {ring_level: set() for ring_level in range(1, D + 1)}
        for ring_level, ring in enumerate(rings, start=1):
            for neighbor in ring:
                children = h3.h3_to_children(neighbor, resolution)
                random.shuffle(list(children))
                for child in children:
                    if child in h3_cells:
                        valid_negative_samples_by_level[ring_level].add(child)
                        if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                            break
                if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                    break

        for point in points:
            positive_points = [p for p in points if p != point]
            if positive_points:
                sampled_positives = random.sample(positive_points, min(K, len(positive_points)))
                for positive_sample in sampled_positives:
                    for ring_level in range(1, D + 1):
                        valid_negative_samples = valid_negative_samples_by_level[ring_level]
                        if valid_negative_samples:
                            sampled_negatives = random.sample(list(valid_negative_samples), min(T, len(valid_negative_samples)))
                            for neg in sampled_negatives:
                                neg_index = h3_cells.index(neg)
                                triplets.append((point, positive_sample, neg_index, ring_level))

    return triplets

def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
    cell_to_points = defaultdict(list)
    h3_cells_set = set(h3_cells)

    for index, cell in enumerate(h3_cells):
        if is_valid_h3_cell(cell):
            cell_to_points[cell].append(index)

    triplets = []
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(generate_triplets_for_cell, cell, points, h3_cells_set, K, T, D, resolution): cell for cell, points in cell_to_points.items()}
        
        # Attach tqdm progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Generating Triplets"):
            triplets.extend(future.result())

    return triplets

def form_triplets():
    combined_df = pd.read_csv(combined_csv_path)
    
    addresses = combined_df['cleaned_address'].tolist()
    lat_lngs = combined_df[['lookup_lat', 'lookup_lng']].values.tolist()
    
    resolution = 10
    h3_cells = generate_h3_cells(lat_lngs, resolution)
    
    K = 1  # Set the desired value for K
    T = 1  # Set the desired value for T
    D = 3  # Set the desired value for D
    triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)
    
    with open(output_pickle_path, 'wb') as f:
        pickle.dump({
            'triplets': triplets,
            'addresses': addresses,
            'lat_lngs': lat_lngs
        }, f)
    
    print(f"Combined data saved to {output_pickle_path}")

if __name__ == "__main__":
    form_triplets()


KeyboardInterrupt: 